# Linear regression

In [2]:
from pyspark.ml.regression import LinearRegression

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

21/07/28 15:48:40 WARN Utils: Your hostname, GBLON1WLZ13699 resolves to a loopback address: 127.0.1.1; using 10.164.16.79 instead (on interface eth2)
21/07/28 15:48:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/28 15:48:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
data_path = '/home/juvid/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/sample_linear_regression_data.txt'

In [7]:
training = spark.read.format("libsvm").load(data_path)

21/07/28 15:48:49 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

Notice that features is a collection of features

In [12]:
a_feature_vector = training.head(1)[0].features

In [19]:
a_feature_vector

SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658})

In [18]:
a_feature_vector.size

10

In [20]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [21]:
res = lr.fit(training)

21/07/28 15:59:06 WARN Instrumentation: [7747e190] regParam is zero, which might cause numerical instability and overfitting.
21/07/28 15:59:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/07/28 15:59:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
21/07/28 15:59:06 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/07/28 15:59:06 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [22]:
res.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [23]:
res.intercept

0.14228558260358093

In [26]:
summary = res.summary

In [27]:
summary.pValues

[0.9927505031240562,
 0.30967074330990396,
 0.3178269194409711,
 0.003972477331573909,
 0.5201486327242175,
 0.16213017210149872,
 0.7102819001865635,
 0.5266812209137877,
 0.46256007153356316,
 0.37825808848978526,
 0.7592692146070568]

In [33]:
summary.meanSquaredError

103.28843028724194

---

Redo with a test/train split

In [34]:
all_data = spark.read.format("libsvm").load(data_path)

21/07/28 16:01:59 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [36]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [37]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                342|
|   mean|-0.6213795488815526|
| stddev| 10.570890894672331|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [38]:
test_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|               159|
|   mean|  2.14599437507661|
| stddev|    9.511326381021|
|    min|-23.51088409032297|
|    max|26.903524792043335|
+-------+------------------+



In [39]:
res = lr.fit(train_data)

21/07/28 16:04:06 WARN Instrumentation: [b40f6157] regParam is zero, which might cause numerical instability and overfitting.


In [40]:
test_preds = res.evaluate(test_data)

In [43]:
test_preds.rootMeanSquaredError

9.926487686438854

Predict on unlabelled data

In [45]:
unlabelled_features = test_data.select('features')

In [48]:
predictions = res.transform(unlabelled_features)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -2.1291459114382367|
|(10,[0,1,2,3,4,5,...| -0.8862321530312158|
|(10,[0,1,2,3,4,5,...|  -2.655639009307706|
|(10,[0,1,2,3,4,5,...| -1.9196123507387894|
|(10,[0,1,2,3,4,5,...| -1.1891806926080375|
|(10,[0,1,2,3,4,5,...|-0.03146160672959908|
|(10,[0,1,2,3,4,5,...|  2.3184435415633255|
|(10,[0,1,2,3,4,5,...| -0.3695935771287562|
|(10,[0,1,2,3,4,5,...| -2.6604384451534977|
|(10,[0,1,2,3,4,5,...|   -1.55571132232169|
|(10,[0,1,2,3,4,5,...|  -0.832984412508025|
|(10,[0,1,2,3,4,5,...|   2.153324408685861|
|(10,[0,1,2,3,4,5,...|  0.4869102719824333|
|(10,[0,1,2,3,4,5,...| -1.9081788549467409|
|(10,[0,1,2,3,4,5,...|  -5.559904148263277|
|(10,[0,1,2,3,4,5,...|  -6.827287507666536|
|(10,[0,1,2,3,4,5,...|  -5.219842229281323|
|(10,[0,1,2,3,4,5,...| 0.07202955377842779|
|(10,[0,1,2,3,4,5,...|   1.430239190394722|
|(10,[0,1,2,3,4,5,...|-0.1327925